[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/riquebueno/topicosaprendizadomaquina/blob/master/Trabalho1TopicosAprendMaq.ipynb)

In [3]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


https://towardsdatascience.com/learn-word2vec-by-implementing-it-in-tensorflow-45641adaf2ac

https://medium.com/deep-math-machine-learning-ai/chapter-9-2-nlp-code-for-word2vec-neural-network-tensorflow-544db99f5334

Será que é possível substituir o GENSIM pelo NLTK?

ok Resolver o problema da diferença entre quantidade de registros do transcript e do main.

#Tópicos Avançados em Aprendizado de Máquina - Olhar esse artigo do Google https://cloud.google.com/blog/products/gcp/problem-solving-with-ml-automatic-document-classification - Antes de tudo fazer uma análise de dados com alguns gráficos - Acho que posso primeiro aplicar o TF-IDF para selecionar as 30 palavras mais importantes de cada documento, ao invés de pegar as 30 primeiras.

In [4]:
#Instalação de bibliotecas
!pip install -U gensim
!pip install -U nltk

    100% |████████████████████████████████| 23.6MB 2.0MB/s 
    100% |████████████████████████████████| 1.4MB 14.6MB/s 
    100% |████████████████████████████████| 133kB 28.2MB/s 
    100% |████████████████████████████████| 4.7MB 7.7MB/s 
    100% |████████████████████████████████| 61kB 26.1MB/s 
    100% |████████████████████████████████| 552kB 25.4MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file
    100% |████████████████████████████████| 1.4MB 12.0MB/s 
  Running setup.py bdist_wheel for nltk ... - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
Successfully built nltk
  Found

In [0]:
#Imports
import pandas as pd
import numpy as np
import nltk as nltk
from sklearn import datasets, linear_model
import gensim, logging

#set up logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [83]:
#Leitura do dataset

#df = pd.read_csv('ted_main.csv')
#df.ratings[0]
df = pd.read_csv('transcripts.csv')
#umTexto = df.transcript[0]
#olhar depois stopwords = nltk.corpus.stopwords.words('portuguese')
#acho que também precisarei fazer stemming: pegar do nltk.stem!!!!!!!!!!!!!!!!!!!!
#http://thalesbertaglia.com/enelvo/sobre/ para normalização
textos = df['transcript']
textos_URL = df['url']
#print(textos[0])
#print(textos_URL[0])


df2 = pd.read_csv('ted_main.csv')
#umTexto = df.transcript[0]
#olhar depois stopwords = nltk.corpus.stopwords.words('portuguese')
#acho que também precisarei fazer stemming: pegar do nltk.stem!!!!!!!!!!!!!!!!!!!!
#http://thalesbertaglia.com/enelvo/sobre/ para normalização
main_tags = df2['tags']
main_URL = df2['url']
#print(main_tags[0])
#print(main_URL[0])

#tabela que vai guardar o texto, os tags e as URLs
#
estruturaFinal=[]

#Existem alguns textos sem transcrições. Irei descartá-los
for indexTextos, textoURL in enumerate(textos_URL):
  for indexMain, linhaMainURL in enumerate(main_URL):
    if textoURL==linhaMainURL:
      estruturaFinal.append([textos[indexTextos], main_tags[indexMain], textoURL])
      break
      
#print(len(estruturaFinal))
print(estruturaFinal[0][0])
print(estruturaFinal[0][1])
print(estruturaFinal[0][2])
#print("===============")
#print(estruturaFinal[100][0])
#print(estruturaFinal[100][1])
#print(estruturaFinal[100][2])
#print("===============")
#print(estruturaFinal[1000][0])
#print(estruturaFinal[1000][1])
#print(estruturaFinal[1000][2])
#print("===============")
#print(estruturaFinal[2466][0])
#print(estruturaFinal[2466][1])
#print(estruturaFinal[2466][2])

textos=[]
for item in estruturaFinal:
  textos.append(item[0])
print(textos[0])

#As 2 estruturas que usarei o tempo todo são estruturaFinal (texto, tag e url) e textos (texto)
print(len(estruturaFinal))

Good morning. How are you?(Laughter)It's been great, hasn't it? I've been blown away by the whole thing. In fact, I'm leaving.(Laughter)There have been three themes running through the conference which are relevant to what I want to talk about. One is the extraordinary evidence of human creativity in all of the presentations that we've had and in all of the people here. Just the variety of it and the range of it. The second is that it's put us in a place where we have no idea what's going to happen, in terms of the future. No idea how this may play out.I have an interest in education. Actually, what I find is everybody has an interest in education. Don't you? I find this very interesting. If you're at a dinner party, and you say you work in education — Actually, you're not often at dinner parties, frankly.(Laughter)If you work in education, you're not asked.(Laughter)And you're never asked back, curiously. That's strange to me. But if you are, and you say to somebody, you know, they sa

In [84]:
#Etapa de pré-processamento
#Preciso dar mais atenção à etapa de pré-processamento

caracteresARemover = "[]?<>\{\}\(\)!@#$.,;:\"\'-_—♫"

textosProcessados = []
#Remoção de caracteres
for texto in textos:
  for i in range(0,len(caracteresARemover)):
    texto = texto.lower().replace(caracteresARemover[i]," ")
  textosProcessados.append(texto)
  
#Split dos textos e eliminação de palavras vazias
palavrasDeTodosOsTextos = []
for texto in textosProcessados:
  palavrasDeUmTexto = texto.split(" ")
  #removendo os strings vazios
  palavrasDeUmTexto = list(filter(None, palavrasDeUmTexto))
  palavrasDeTodosOsTextos.append(palavrasDeUmTexto)
  
#PENDENTE - Aplicar TF-IDF
#................
#................
#................

# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in palavrasDeTodosOsTextos:
    for token in text:
        frequency[token] += 1  

# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > 1] for text in palavrasDeTodosOsTextos]

#Preciso eliminar as linhas do processed_corpus que tem menos de NUM_COLUNAS palavras. Esse valor será ajustado para 300
NUM_COLUNAS=300
for index, linha in enumerate(processed_corpus):
  if len(linha)<NUM_COLUNAS:
    del processed_corpus[index]
    del estruturaFinal[index]
    del textos[index]#acho que nao precisa pq nao usarei mais essa estrutura


print(len(processed_corpus))
print(len(estruturaFinal))#texto, tag e url
print(processed_corpus[0])

2432
2432
['good', 'morning', 'how', 'are', 'you', 'laughter', 'it', 's', 'been', 'great', 'hasn', 't', 'it', 'i', 've', 'been', 'blown', 'away', 'by', 'the', 'whole', 'thing', 'in', 'fact', 'i', 'm', 'leaving', 'laughter', 'there', 'have', 'been', 'three', 'themes', 'running', 'through', 'the', 'conference', 'which', 'are', 'relevant', 'to', 'what', 'i', 'want', 'to', 'talk', 'about', 'one', 'is', 'the', 'extraordinary', 'evidence', 'of', 'human', 'creativity', 'in', 'all', 'of', 'the', 'presentations', 'that', 'we', 've', 'had', 'and', 'in', 'all', 'of', 'the', 'people', 'here', 'just', 'the', 'variety', 'of', 'it', 'and', 'the', 'range', 'of', 'it', 'the', 'second', 'is', 'that', 'it', 's', 'put', 'us', 'in', 'a', 'place', 'where', 'we', 'have', 'no', 'idea', 'what', 's', 'going', 'to', 'happen', 'in', 'terms', 'of', 'the', 'future', 'no', 'idea', 'how', 'this', 'may', 'play', 'out', 'i', 'have', 'an', 'interest', 'in', 'education', 'actually', 'what', 'i', 'find', 'is', 'everybody'

In [69]:
#Recuperação dos modelos de embedding

import gensim.downloader as api
modelGlove = api.load("glove-wiki-gigaword-100")
#modelWord2Vec = api.load("word2vec-google-news-300")
#modelFastText = api.load("fasttext-wiki-news-subwords-300")

#como verificar se uma palavra existe em um Word2VecKeyedVectors?
#vector = modelGlove["cant"]  # numpy vector of a word
#vector.shape
#vector

2018-10-17 19:00:39,264 : INFO : Creating /root/gensim-data


[==================================================] 100.0% 128.1/128.1MB downloaded


2018-10-17 19:01:11,759 : INFO : glove-wiki-gigaword-100 downloaded
2018-10-17 19:01:11,762 : INFO : loading projection weights from /root/gensim-data/glove-wiki-gigaword-100/glove-wiki-gigaword-100.gz
2018-10-17 19:02:09,986 : INFO : loaded (400000, 100) matrix from /root/gensim-data/glove-wiki-gigaword-100/glove-wiki-gigaword-100.gz


In [0]:
#Criação da matriz de embeddings

#agora vou converter a tabela de textos processed_corpus para uma
#tabela de vetores. Substitui cada vetor pela média de seus valores.
#Assim, a célula passou a ser um número e não um vetor.

numeroLinhas=len(processed_corpus)
numeroColunas=10000

#matrizDeVetores=[]
#matrizDeVetores=np.empty([len(processed_corpus), 10000])
matrizDeVetores=np.empty((numeroLinhas, numeroColunas))

for index, linha in enumerate(processed_corpus):
  print(index)
  #linhaDeVetores=[]
  #linhaDeVetores=np.empty((1, numeroColunas))
  coluna=0
  for indicePalavra, palavra in enumerate(linha):
    try:
      vetorDaPalavra = modelGlove[palavra]
      
      #Sugestão para fazer a média do embedding
      #http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/
      mediaDoVetorDaPalavra=np.mean(vetorDaPalavra)
      #linhaDeVetores.append(vetorDaPalavra)
      #linhaDeVetores.append(mediaDoVetorDaPalavra)
      matrizDeVetores[index][coluna]=mediaDoVetorDaPalavra
      coluna=coluna+1
    except:
      #print(palavra)
      #print("Unexpected error:", sys.exc_info()[0])
      continue
    
  #matrizDeVetores.append(linhaDeVetores)
  #matrizDeVetores[index] = linhaDeVetores

print(len(matrizDeVetores[0]))
#print(matrizDeVetores)

In [127]:
print(matrizDeVetores.shape)

#existe algum linha sem nenhuma palavra?
print(len(matrizDeVetores))

for elem in matrizDeVetores:
  if len(elem)==0:
    print(str(index) + "vazioooooooo!")

(2432, 10000)
2432


In [118]:
total=0
min=10000000
max=-1
for item in matrizDeVetores:
  total=total+len(item)
  if len(item)<min:
    min=len(item)
  if len(item)>max:
    max=len(item)
media=total/len(matrizDeVetores)
print("media " + str(media))
print("min " + str(min))
print("max " + str(max))


media 2141.0793585526317
min 304
max 9363


In [128]:
print(matrizDeVetores[0])
print(matrizDeVetores[1])
print(matrizDeVetores[1][0])

[-5.47338724e-02 -1.56281404e-02 -7.28304908e-02 ...  7.11730072e-67
  3.24249373e-86  1.17687697e-47]
[-2.29197294e-002 -2.67957300e-002 -4.29836139e-002 ...  1.05206362e-153
  1.31505368e-047  1.25767118e-076]
-0.022919729351997375


In [0]:
#Vou limitar matrizDeVetores para 300 colunas


In [129]:
#Criação do vetor de classificações Y

#preciso gerar o código que gerará o vetor Y. Primeiro vou trabalhar com apenas
#o primeiro tag. Depois trabalharei com multi-label.
#df = pd.read_csv('ted_main.csv')
#df.ratings[0]
Y=[]
#tags = df['tags']

tags=[]
for item in estruturaFinal:
  tags.append(item[1])#0: texto, 1:tags, 2: url

for linha in tags:
  linha=linha.lower()
  linha=linha.replace("[","")
  linha=linha.replace("]","")
  linha=linha.replace("'","")
  linhaSplitada=linha.split(",")
  primeiroTagDaLinha=linhaSplitada[0]
  Y.append(primeiroTagDaLinha)
print(len(Y))
print(Y)
mySet=set(Y)
print(len(mySet))#numero de tags diferentes
print(mySet)
print(tags)

2432
['children', 'alternative energy', 'computers', 'macarthur grant', 'africa', 'business', 'christianity', 'architecture', 'god', 'christianity', 'activism', 'ted prize', 'ted prize', 'demo', 'children', 'entertainment', 'creativity', 'macarthur grant', 'dna', 'business', 'business', 'collaboration', 'business', 'cognitive science', 'culture', 'climate change', 'astronomy', 'business', 'business', 'ted brain trust', 'business', 'cancer', 'aging', 'alternative energy', 'africa', 'business', 'business', 'nasa', 'climate change', 'aids', 'ted prize', 'ted prize', 'culture', 'culture', 'ted brain trust', 'choice', 'ted brain trust', 'business', 'culture', 'culture', 'business', 'aids', 'anthropology', 'activism', 'astronomy', 'collaboration', 'cities', 'business', 'macarthur grant', 'macarthur grant', 'code', 'choice', 'africa', 'ted prize', 'activism', 'africa', 'ted brain trust', 'dna', 'brain', 'business', 'dna', 'ted brain trust', 'alternative energy', 'architecture', 'art', 'entert

In [0]:
print(Y[0])

In [130]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# define example
#data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
data = Y

values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)

Y = integer_encoded
print(Y)

['children' 'alternative energy' 'computers' ... 'ai' 'internet' 'cities']
[ 56  13  71 ...   9 136  60]


In [0]:
#Aplicação de técnicas de classificação

#Aplicando Regressão Logística
#Y=['A','A','B']
X=matrizDeVetores

from sklearn import linear_model
logreg = linear_model.LogisticRegression(C=1.0)
logreg.fit(X, Y)
print("Training set score: %f" % logreg.score(X, Y))

#Aplicando Redes Neurais
#from sklearn.neural_network import MLPClassifier
#mlp = MLPClassifier(hidden_layer_sizes=(50,), max_iter=10, alpha=1e-4,
#                    solver='sgd', verbose=10, tol=1e-4, random_state=1,
#                    learning_rate_init=.1)
#mlp.fit(X, Y)
#print("Training set score: %f" % mlp.score(X, Y))

In [0]:
#Sugestão: comparar com bag of words

In [0]:
>>> from sklearn.datasets import load_iris
>>> from sklearn import tree
>>> iris = load_iris()
>>> clf = tree.DecisionTreeClassifier()
>>> clf = clf.fit(iris.data, iris.target)

# Rascunhos


In [0]:
#palavrasDeTodosOsTextos
#frequency
#processed_corpus

In [0]:
#from gensim import corpora

#dictionary = corpora.Dictionary(processed_corpus)
#print(dictionary)
#45427 unique tokens: ["'30s", '15', '16', '1930s', '19th']...) from 2467 documents

In [0]:
#print(dictionary.token2id)

In [0]:
#preciso implementar o seguinte: pegar as 30 primeiras palavras de cada texto e 
#pegar o vetor delas (farei isso para cada tipo de embedding).



model = gensim.models.Word2Vec(processed_corpus, min_count=1)

In [0]:
#print(model)
#print(model.wv.vocab)
model.most_similar("music")

In [0]:
modelGlove.train(["henrique", "heloisa"])

In [0]:
  modelGlove.most_similar(positive=['woman', 'king'], negative=['man'])

In [0]:
api.info()

In [0]:
QTD_TEXTOS=len(Y)
QTD_PALAVRAS=30

In [0]:
#construção da tabela entrada da regressão X
Y
X=np.array(ndmin=2)
numeroDaLinha=-1
for text in texts:
  numeroDaLinha+=1
  for i in range(QTD_PALAVRAS):#vou pegar as 30 primeiras palavras de cada texto
    palavra = text[i]
    vetorDaPalavra = modelo[palavra]
    X[numeroDaLinha][i]=vetorDaPalavra
    

In [0]:
#nltk.download()

In [0]:
#https://radimrehurek.com/gensim/models/keyedvectors.html é melhor usar full model 
# quando You need the full model to train or update vectors.
#The reason for separating the trained vectors into KeyedVectors is that if you don’t need 
#the full model state any more (don’t need to continue training), the state can discarded, 
#resulting in a much smaller and faster object that can be mmapped for lightning fast loading 
#and sharing the vectors in RAM between processes:



# Load Google's pre-trained Word2Vec model.
#depreciado
#model = gensim.models.Word2Vec.load_word2vec_format('./model/GoogleNews-vectors-negative300.bin', binary=True)  
from gensim.test.utils import datapath
from gensim.models import KeyedVectors
#Vectors exported by the Facebook and Google tools do not support further training, 
#but you can still load them into KeyedVectors.
wv_from_text = KeyedVectors.load_word2vec_format(datapath('word2vec_pre_kv_c'), binary=False)  # C text format


In [0]:
from gensim.models import Word2Vec
model = Word2Vec(common_texts, size=100, window=5, min_count=1, workers=4)
model.train()

In [0]:
#model = gensim.models.Word2Vec(processed_corpus, min_count=1)
model = gensim.models.Word2Vec(['carro','barco'], min_count=1)

In [0]:
model=gensim.models.Word2Vec.load_word2vec_format()#depreciado

In [0]:
import gensim.downloader as api
api.info()

In [0]:


word_vectors = api.load("glove-wiki-gigaword-100")

In [0]:
# load an example dataset
from vega_datasets import data
cars = data.cars()

# plot the dataset, referencing dataframe column names
import altair as alt
alt.Chart(cars).mark_bar().encode(
  x=alt.X('Miles_per_Gallon', bin=True),
  y='count()',
)

In [0]:
#https://github.com/RaRe-Technologies/gensim
#https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/gensim%20Quick%20Start.ipynb



raw_corpus = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",              
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

# Create a set of frequent words
stoplist = set('for a of the and to in'.split(' '))
# Lowercase each document, split it by white space and filter out stopwords
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in raw_corpus]

# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]
processed_corpus


In [0]:




#These datasets contain information about all audio-video recordings of TED Talks uploaded to the official TED.com website until September 21st, 2017. The TED main dataset contains information about all talks including number of views, number of comments, descriptions, speakers and titles. The TED transcripts dataset contains the transcripts for all talks available on TED.com.
#There are two CSV files:
#ted_main.csv - Contains data on actual TED Talk metadata and TED Talk speakers.
#transcripts.csv - Contains transcript and URL information for TED Talks

